In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression # 0.32
# from sklearn.linear_model import MultiTaskLassoCV # too slow
# from sklearn.linear_model import Lasso # too slow
from sklearn.linear_model import Ridge # best one so far
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNet # haven't tried, worth trying
from sklearn.kernel_ridge import KernelRidge # haven't tried, similar to SVR
# from sklearn.ensemble import RandomForestRegressor # poor performance 0.05 with word2vec
# from sklearn.cross_decomposition import PLSRegression # worse than Ridge
# from sklearn.cross_decomposition import PLSCanonical # haven't tried
# from sklearn.svm import SVR # haven't tried
# from sklearn.svm import SVC # haven't tried
# from sklearn.svm import LinearSVC # haven't tried
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split

In [2]:
# d: description, t: tag, f: 1000 feature, fi: 2048 intermediate features, bow: bag of word, w2v: word 2 vector
# w2v poor performance
d_train_bow = np.load('preprocessing/bagOfWord_train_description.npy')
# d_train_w2v = np.load('preprocessing/word2vec_train_description.npy')
t_train_bow = np.load('preprocessing/bagOfWord_train_tag.npy')
# t_train_w2v = np.load('preprocessing/word2vec_train_tag.npy')
f_train = np.load('preprocessing/word2vec_train_1000feature.npy')
fi_train = np.load('preprocessing/word2vec_train_2048feature.npy')

print(d_train_bow.shape)
# print(d_train_w2v.shape)
print(t_train_bow.shape)
# print(t_train_w2v.shape)
print(f_train.shape)
print(fi_train.shape)
print()

d_test_bow = np.load('preprocessing/bagOfWord_test_description.npy')
# d_test_w2v = np.load('preprocessing/word2vec_test_description.npy')
t_test_bow = np.load('preprocessing/bagOfWord_test_tag.npy')
# t_test_w2v = np.load('preprocessing/word2vec_test_tag.npy')
f_test = np.load('preprocessing/word2vec_test_1000feature.npy')
fi_test = np.load('preprocessing/word2vec_test_2048feature.npy')

print(d_test_bow.shape)
# print(d_test_w2v.shape)
print(t_test_bow.shape)
# print(t_test_w2v.shape)
print(f_test.shape)
print(fi_test.shape)
print()

# no idf bow
d_train_bow_no_idf = np.load('preprocessing/bagOfWord_train_description_no_idf.npy')
t_train_bow_no_idf = np.load('preprocessing/bagOfWord_train_tag_no_idf.npy')
d_test_bow_no_idf = np.load('preprocessing/bagOfWord_test_description_no_idf.npy')
t_test_bow_no_idf = np.load('preprocessing/bagOfWord_test_tag_no_idf.npy')

print(d_train_bow_no_idf.shape)
print(t_train_bow_no_idf.shape)
print(d_test_bow_no_idf.shape)
print(t_test_bow_no_idf.shape)
print()

# intersection idf and no idf
d_train_bow_intersection = np.load('preprocessing/bagOfWord_intersection_train_description.npy')
t_train_bow_intersection = np.load('preprocessing/bagOfWord_intersection_train_tag.npy')
d_test_bow_intersection = np.load('preprocessing/bagOfWord_intersection_test_description.npy')
t_test_bow_intersection = np.load('preprocessing/bagOfWord_intersection_test_tag.npy')

print(d_train_bow_intersection.shape)
print(t_train_bow_intersection.shape)
print(d_test_bow_intersection.shape)
print(t_test_bow_intersection.shape)
print()

d_train_bow_intersection_no_idf = np.load('preprocessing/bagOfWord_intersection_train_description_no_idf.npy')
t_train_bow_intersection_no_idf = np.load('preprocessing/bagOfWord_intersection_train_tag_no_idf.npy')
d_test_bow_intersection_no_idf = np.load('preprocessing/bagOfWord_intersection_test_description_no_idf.npy')
t_test_bow_intersection_no_idf = np.load('preprocessing/bagOfWord_intersection_test_tag_no_idf.npy')

print(d_train_bow_intersection_no_idf.shape)
print(t_train_bow_intersection_no_idf.shape)
print(d_test_bow_intersection_no_idf.shape)
print(t_test_bow_intersection_no_idf.shape)
print()

(10000, 7077)
(10000, 102)
(10000, 1000)
(10000, 2048)

(2000, 7077)
(2000, 102)
(2000, 1000)
(2000, 2048)

(10000, 7077)
(10000, 102)
(2000, 7077)
(2000, 102)

(10000, 2776)
(10000, 102)
(2000, 2776)
(2000, 102)

(10000, 2776)
(10000, 102)
(2000, 2776)
(2000, 102)



In [ ]:
trainX = np.concatenate((t_train_bow_intersection, f_train, fi_train), axis=1)
print(trainX.shape)
trainY = d_train_bow_intersection
print(trainY.shape)
testX = np.concatenate((t_test_bow_intersection, f_test, fi_test), axis=1)
print(testX.shape)
query = d_test_bow_intersection

In [4]:
def output_prediction(testY, filename):
    distance = cdist(query, testY, 'euclidean')
    f = open(filename, "w")
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i in range(2000):
        test_dist_idx = list(np.argsort(distance[i]))
        top_20 = test_dist_idx[:20]
        row = ["%d.jpg" % i for i in top_20]
        f.write("%d.txt,%s\n" % (i, " ".join(row)))
    f.close()
    print("Output written!")

In [ ]:
testY = LinearRegression(n_jobs=-1).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/linear_regression_submission.csv')

In [ ]:
# too slow
lasso = Lasso(alpha=1, selection='random').fit(trainX, trainY)

In [ ]:
# too slow
lasso1 = MultiTaskLassoCV(alphas=[0.01], max_iter=100, selection='random', n_jobs=-1).fit(trainX, trainY)
print(lasso1.alpha_)

In [ ]:
# too slow
lasso = Lasso(alpha=1, selection='random').fit(trainX, trainY)

In [ ]:
# too slow
new_alpha = {0.0001:[0.0001, 0.0002, 0.0003, 0.0004],
            0.0005:[0.0005, 0.0006, 0.0007, 0.0008],
            0.001:[0.001, 0.002, 0.003, 0.004],
            0.005:[0.005,0.006,0.007,0.008],
            0.01:[0.01,0.02,0.03,0.04],
            0.05:[0.05,0.06,0.07,0.08],
            0.1:[0.1,0.2,0.3,0.4],
            0.5:[0.5,0.6,0.7,0.8]}
lasso2 = MultiTaskLassoCV(alphas=new_alpha[lasso1.alpha_], n_jobs=-1).fit(trainX, trainY)
print(lasso2.alpha_)

In [ ]:
# too slow
testY = lasso2.predict(testX)
output_prediction( testY, 'output/lasso_regression_alpha'+str(lasso2.alpha_)+'submission.csv')

In [ ]:
testY = Ridge(alpha=1).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha1.csv')

In [ ]:
testY = Ridge(alpha=0.1).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha0.1_submission.csv')
# worse than alpha=1

In [ ]:
testY = Ridge(alpha=2).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha2_submission.csv')
# slightly better than alpha=1

In [ ]:
testY = Ridge(alpha=5).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha5_submission.csv')
# slightly better than alpha=2

In [ ]:
testY = Ridge(alpha=10).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha10_submission.csv')
# slightly better than alpha=5

In [ ]:
testY = Ridge(alpha=15).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha15_submission.csv')
# slightly better than alpha=10

In [ ]:
testY = Ridge(alpha=20).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha20_submission.csv')
# slightly better than alpha=15

In [ ]:
testY = Ridge(alpha=30).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha30_submission.csv')
# slightly better than alpha=20

In [ ]:
testY = Ridge(alpha=50).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha50_submission.csv')
# slightly better than alpha=30

In [ ]:
testY = Ridge(alpha=100).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha100_submission.csv')
# slightly better than alpha=50, the best so far

In [ ]:
testY = Ridge(alpha=200).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha200_submission.csv')
# slightly worse than alpha=100

In [ ]:
testY = Ridge(alpha=150).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha150_submission.csv')
# slightly worse than alpha=100

In [ ]:
testY = Ridge(alpha=125).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha125_submission.csv')
# slightly worse than alpha=100

In [ ]:
testY = Ridge(alpha=75).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha75_submission.csv')
# slightly worse than alpha=100

In [14]:
trainX = np.concatenate((t_train_bow_intersection, f_train, fi_train), axis=1)
print(trainX.shape)
trainY = d_train_bow_intersection
print(trainY.shape)
testX = np.concatenate((t_test_bow_intersection, f_test, fi_test), axis=1)
print(testX.shape)
query = d_test_bow_intersection

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

(10000, 3150)
(10000, 2776)
(2000, 3150)
0.28641282293870873
0.29023530482485244
0.2887009392175183
0.30067964130853636
0.3037845207051722
0.302127857744366
0.3036424836379612
0.306276656783759
0.30454101600335604


In [18]:
trainX = np.concatenate((t_train_bow_intersection_no_idf, f_train, fi_train), axis=1)
print(trainX.shape)
trainY = d_train_bow_intersection_no_idf
print(trainY.shape)
testX = np.concatenate((t_test_bow_intersection_no_idf, f_test, fi_test), axis=1)
print(testX.shape)
query = d_test_bow_intersection_no_idf

def test(alpha):
    s = 0
    copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=457)
    ridge = Ridge(alpha=alpha).fit(copyX,copyY)
    s += ridge.score(devX, devY)

    copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=27)
    ridge = Ridge(alpha=alpha).fit(copyX,copyY)
    s += ridge.score(devX, devY)

    copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=4778)
    ridge = Ridge(alpha=alpha).fit(copyX,copyY)
    s += ridge.score(devX, devY)
    s /= 3
    print(s)
    return s
alphas = [90, 95, 100, 105, 110]
max_val = 0
max_a = 0
for a in alphas:
    temp = test(a)
    if temp > max_val:
        max_val = temp
        max_a = a

print()
print(max_val, max_a)

(10000, 3150)
(10000, 2776)
(2000, 3150)
0.33811885469527364
0.3382689393184896
0.33834491868620037
0.33835786303376914
0.33831700763524014

0.33835786303376914 105


In [5]:
trainX = np.concatenate((t_train_bow_intersection, f_train, fi_train), axis=1)
print(trainX.shape)
trainY = d_train_bow_intersection
print(trainY.shape)
testX = np.concatenate((t_test_bow_intersection, f_test, fi_test), axis=1)
print(testX.shape)
query = d_test_bow_intersection
testY = Ridge(alpha=110).fit(trainX, trainY).predict(testX)
output_prediction(testY, 'output/ridge_regression_alpha110_intersection_submission.csv')
# slightly worse than alpha=100

(10000, 3150)
(10000, 2776)
(2000, 3150)
Output written!


In [15]:
trainX = np.concatenate((t_train_bow_intersection_no_idf, f_train, fi_train), axis=1)
print(trainX.shape)
trainY = d_train_bow_intersection_no_idf
print(trainY.shape)
testX = np.concatenate((t_test_bow_intersection_no_idf, f_test, fi_test), axis=1)
print(testX.shape)
query = d_test_bow_intersection_no_idf

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=110).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=110).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=110).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=125).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=125).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=125).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

(10000, 3150)
(10000, 2776)
(2000, 3150)
0.342549805717546
0.3424544788603109
0.33910741308968245
0.34214257766717443
0.3420135844849383
0.3386187833761429
0.34097577748783514
0.34081869708526
0.33738001498001374


In [12]:
trainX = np.concatenate((t_train_bow, f_train, fi_train), axis=1)
print(trainX.shape)
trainY = d_train_bow
print(trainY.shape)
testX = np.concatenate((t_test_bow, f_test, fi_test), axis=1)
print(testX.shape)
query = d_test_bow

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

(10000, 3150)
(10000, 7077)
(2000, 3150)
0.2582717181697155
0.2622434856998263
0.260291936128236
0.27222345706680734
0.27559833951732327
0.2734908641883313
0.27533492815896005
0.27829532219716996
0.2761078573191877


In [13]:
trainX = np.concatenate((t_train_bow_no_idf, f_train, fi_train), axis=1)
print(trainX.shape)
trainY = d_train_bow_no_idf
print(trainY.shape)
testX = np.concatenate((t_test_bow_no_idf, f_test, fi_test), axis=1)
print(testX.shape)
query = d_test_bow_no_idf

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=50).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=100).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=785)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

copyX, devX, copyY, devY = train_test_split(trainX, trainY, test_size=0.2, random_state=349)
ridge = Ridge(alpha=150).fit(copyX,copyY)
print(ridge.score(devX, devY))

(10000, 3150)
(10000, 7077)
(2000, 3150)
0.32902569456379316
0.3290880444550217
0.3256057037564996
0.33564799996316125
0.3356314822945693
0.3320555819806721
0.334210158250084
0.3341370092259414
0.33047073305774544
